## Figure on inference in L5PC model on simulated data

In [ ]:
%run -i common.ipynb

import json
import os
path1 = os.getcwd()
# change the current working directory to path
path2 = '/home/pedro/repos/lfi-models/lfimodels/l5pc/'
os.chdir(path2)

import lfimodels.l5pc.utils as utils

from lfimodels.l5pc.L5Pyramidal import L5Pyramidal
from lfimodels.l5pc.L5PCStats import L5PCStats

# FIGURE and GRID
FIG_HEIGHT_MM = 90
FIG_WIDTH_MM = FIG_WIDTH_MM  # set in common notebook to a default value for all figures
FIG_N_ROWS = 2
ROW_1_NCOLS = 2
ROW_1_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_1_WIDTH_COL_1_MM = 0.5*FIG_WIDTH_MM / ROW_1_NCOLS
ROW_1_WIDTH_COL_2_MM = 1.5*FIG_WIDTH_MM / ROW_1_NCOLS
ROW_2_NCOLS = 2
ROW_2_HEIGHT_MM = FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = 0.5*FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = 0.5*FIG_WIDTH_MM / ROW_2_NCOLS

W_FACT = 0.97
H_FACT = 0.97

PATH_DROPBOX_FIGS = PATH_DROPBOX + 'figs/'

# PATHS
PANEL_A = PATH_DROPBOX_FIGS + 'fig_l5pc_a.svg'
PANEL_B = PATH_DROPBOX_FIGS + 'fig_l5pc_b.svg'
PANEL_C = PATH_DROPBOX_FIGS + 'fig_l5pc_c.svg'
PANEL_D = PATH_DROPBOX_FIGS + 'fig_l5pc_d.svg'

## save figures or not

In [ ]:
save_fig = 1 # 1: save figures

### load data

In [ ]:
feature_configs = json.load(open('config/features.json'))
efel_feature_name_ls = []
for protocol_name, locations in sorted(feature_configs.items()):
    for location, features in sorted(locations.items()):
        for efel_feature_name, meanstd in sorted(features.items()):
            efel_feature_name_ls.append(efel_feature_name)            


true_params, labels_params = utils.obs_params()

seed1 = 2
seed = 1

obs = utils.syn_obs_data(params=true_params,seed=seed1)
obs_stats = utils.syn_obs_stats(data=obs,params=true_params)

m = L5Pyramidal(seed=seed)
p = utils.prior(true_params=true_params,labels_params=labels_params,seed=seed)
s = L5PCStats(seed=seed)
  
# SNPE parameters
n_components = 1
n_sims = 50000
n_rounds = 1
svi = False
if svi:
    svi_flag = '_svi'
else:
    svi_flag = '_nosvi'


# change the current working directory back to notebook path
os.chdir(path1)

# filename1 = './results/l5pc_run_4_param10'+\
# svi_flag+'_ncomp'+str(n_components)+'_nsims'+str(n_sims*n_rounds)+'_snpe.pkl'
# filename2 = './results/l5pc_run_4_param10'+\
# svi_flag+'_ncomp'+str(n_components)+'_nsims'+str(n_sims*n_rounds)+'_snpe_res.pkl'
# filename1 = './results/l5pc_run_5_param10'+svi_flag+'.pkl'
# filename2 = './results/l5pc_run_5_param10'+svi_flag+'_res.pkl'
# log, train_data, posterior = io.load_pkl(filename1)
# res = io.load(filename2)

filename3 = '../l5pc/results/l5pc_run_5_param10_nosvi_multi_sumstats_v3.pkl'
_, posterior_ls, _ = io.load_pkl(filename3)
posterior = posterior_ls[0]

### panel A

In [ ]:
os.chdir(path1)

In [ ]:
import neurom.viewer

# matplotlib takes figsize specified as inches
# in common SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(W_FACT*ROW_1_WIDTH_COL_1_MM), SCALE_IN*mm2inches(H_FACT*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)
    
    neurom.viewer.draw(neurom.load_neuron(path2+'/morphology/C060114A7.asc'),color=COL['GT']);
    plt.axis('off');
    plt.title('')

    if save_fig:
        plt.savefig(PANEL_A, facecolor='None', transparent=True)  # the figure is saved as svg
        plt.close()
        svg(PANEL_A)
    else:
        plt.show()

### panel B

In [ ]:
# matplotlib takes figsize specified as inches
# in common SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(W_FACT*ROW_2_WIDTH_COL_1_MM), SCALE_IN*mm2inches(H_FACT*ROW_2_HEIGHT_MM))

def plot_responses(responses,color):
    fig, axes = plt.subplots(len(responses), figsize=fig_inches)
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        axes[index].plot(response['time'], response['voltage'], color = color, label=resp_name)
        resp_name = resp_name[0:-2]+'\n (mV)'
        resp_name = resp_name.replace('.', '\n')
        axes[index].set_ylabel(resp_name, multialignment='center')
        axes[index].locator_params(axis='x', nbins=4)
        if index!=2 and index!=5:
            axes[index].set_xticks([])
        if index==5:
            axes[index].set_xlabel('time (ms)')

with mpl.rc_context(fname=MPL_RC):
    plot_responses(obs['data'],color = COL['GT'])    
#     plt.show()
    
    if save_fig:
        plt.savefig(PANEL_B, facecolor='None', transparent=True)  # the figure is saved as svg
        plt.close()
        svg(PANEL_B)
    else:
        plt.show()

### panel C

In [ ]:
prior_min = p.lower
prior_max = p.upper

prior_lims = np.concatenate((prior_min.reshape(-1,1),
                             prior_max.reshape(-1,1)),
                            axis=1)

labels_params1 = ['param '+str(i) for i in range(1,len(labels_params)+1)]
    
# matplotlib takes figsize specified as inches
# in common SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(W_FACT*ROW_1_WIDTH_COL_2_MM), SCALE_IN*mm2inches(H_FACT*2.29*ROW_1_HEIGHT_MM))

with mpl.rc_context(fname=MPL_RC):
    fig = plt.figure(figsize=fig_inches)
    
    plot_pdf(posterior, lims=prior_lims,gt=true_params, labels_params=labels_params1,
             figsize=fig_inches,fontscale=0.5,ticks = True, col2=COL['SNPE'])

    if save_fig:
        plt.savefig(PANEL_C, facecolor='None', transparent=True)  # the figure is saved as svg
        plt.close()
        svg(PANEL_C)
    else:
        plt.show()

### panel D

In [ ]:
mn_post = posterior.xs[np.argmax(posterior.a)].m
x_snpe = m.gen_single(mn_post)

# matplotlib takes figsize specified as inches
# in common SCALE_IN and the conversion function are defined
fig_inches = (SCALE_IN*mm2inches(W_FACT*ROW_2_WIDTH_COL_2_MM), SCALE_IN*mm2inches(H_FACT*ROW_2_HEIGHT_MM))

def plot_responses(responses,color):
    fig, axes = plt.subplots(len(responses), figsize=fig_inches)
    for index, (resp_name, response) in enumerate(sorted(responses.items())):
        axes[index].plot(response['time'], response['voltage'], color = color, label=resp_name)
        resp_name = resp_name[0:-2]+'\n (mV)'
        resp_name = resp_name.replace('.', '\n')
        axes[index].set_ylabel(resp_name, multialignment='center')
        axes[index].locator_params(axis='x', nbins=4)
        if index!=2 and index!=5:
            axes[index].set_xticks([])
        if index==5:
            axes[index].set_xlabel('time (ms)')

with mpl.rc_context(fname=MPL_RC):
    plot_responses(x_snpe['data'],color = COL['SNPE'])    
    
    if save_fig:
        plt.savefig(PANEL_D, facecolor='None', transparent=True)  # the figure is saved as svg
        plt.close()
        svg(PANEL_D)
    else:
        plt.show()

### supplementary figure with comparison between LFI and IBEA

In [ ]:
# # number of simulations for same parameter set
# num_rep = 10

# params_prior_mat = p.gen(num_rep)

# # reject samples outside the prior box
# num_rep1 = 5*num_rep
# samp_post = posterior.gen(num_rep1)
# ind = (samp_post > prior_min) & (samp_post < prior_max)
# samp_post = samp_post[np.prod(ind,axis=1)==1]

# x_snpe_ls = []
# x_snpe_samp_ls = []
# # x_ibea_ls = []
# x_prior_ls = []
# sum_stats_snpe_ls = []
# sum_stats_snpe_samp_ls = []
# # sum_stats_ibea_ls = []
# sum_stats_prior_ls = []
# for rep in range(num_rep):
#     x_snpe = m.gen_single(mn_post)
#     x_snpe_ls.append(x_snpe)

#     sum_stats_snpe = s.calc([x_snpe])[0]
#     sum_stats_snpe_ls.append(sum_stats_snpe)

#     x_snpe_samp = m.gen_single(samp_post[rep,:])
#     x_snpe_samp_ls.append(x_snpe_samp)

#     sum_stats_snpe_samp = s.calc([x_snpe_samp])[0]
#     sum_stats_snpe_samp_ls.append(sum_stats_snpe_samp)

# #     x_ibea = m.gen_single(halloffame[0])
# #     x_ibea_ls.append(x_ibea)

# #     sum_stats_ibea = s.calc([x_ibea])[0]
# #     sum_stats_ibea_ls.append(sum_stats_ibea)

#     x_prior = m.gen_single(params_prior_mat[rep,:])
#     x_prior_ls.append(x_prior)

#     sum_stats_prior = s.calc([x_prior])[0]
#     sum_stats_prior_ls.append(sum_stats_prior)

In [ ]:
# sum_stats_snpe_mat = np.asarray(sum_stats_snpe_ls)
# # sum_stats_ibea_mat = np.asarray(sum_stats_ibea_ls)
# sum_stats_snpe_samp_mat = np.asarray(sum_stats_snpe_samp_ls)
# sum_stats_prior_mat = np.asarray(sum_stats_prior_ls)

# ######################################
# n_summary_stats = len(obs_stats[0])
# xx = np.linspace(1,n_summary_stats,n_summary_stats).astype(int)
# XX = np.tile(xx,(num_rep,1))

# ######################################
# yy_samp = sum_stats_snpe_samp_mat-obs_stats
# yy_samp_err = np.nanstd(sum_stats_snpe_samp_mat-obs_stats,axis=0)
# yy_snpe = sum_stats_snpe_mat-obs_stats
# # yy_ibea = sum_stats_ibea_mat-obs_stats
# yy_prior = sum_stats_prior_mat-obs_stats

# ######################################
# ratio_snpe_percentls = np.nanpercentile(yy_snpe/yy_samp_err,[25,50,75],axis=0)
# ratio_snpe_median = ratio_snpe_percentls[1]
# median_ord = np.argsort(ratio_snpe_median)

# # ratio_ibea_percentls = np.nanpercentile(yy_ibea/yy_samp_err,[25,50,75],axis=0)

# ######################################

# ## matplotlib takes figsize specified as inches
# # in common SCALE_IN and the conversion function are defined
# fig_inches = (SCALE_IN*mm2inches(FIG_WIDTH_MM), SCALE_IN*mm2inches(FIG_WIDTH_MM))

# SUPP = 'supp_fig_l5pc'  # name for appendix figure
# SUPP_FIG_SVG = PATH_DROPBOX_FIGS + SUPP + '.svg'
# SUPP_FIG_PDF = PATH_DROPBOX_FIGS + SUPP + '.pdf'

# with mpl.rc_context(fname=MPL_RC):
#     fig = plt.figure(figsize=fig_inches)

#     plt.fill_between(xx,ratio_snpe_percentls[0][median_ord],ratio_snpe_percentls[2][median_ord],
#                      facecolor=COL['SNPE'], alpha=0.3)
#     plt.plot(xx,ratio_snpe_median[median_ord],color = COL['SNPE'], lw=2,label='SNPE')

# #     plt.fill_between(xx,ratio_ibea_percentls[0][median_ord],ratio_ibea_percentls[2][median_ord],
# #                      facecolor=COL['IBEA'], alpha=0.3)
# #     plt.plot(xx,ratio_ibea_percentls[1][median_ord],color = COL['IBEA'], lw=2,label='IBEA')

#     plt.xlabel('features')
#     plt.ylabel(r'$\frac{f^* - f}{\sigma_{SNPE}}$')

#     plt.plot(xx,np.ones(n_summary_stats),'--k')
#     plt.plot(xx,-np.ones(n_summary_stats),'--k')

#     ax = plt.gca()
#     plt.locator_params(axis='x', nbins=5)
#     plt.locator_params(axis='y', nbins=4)
#     ax.legend(loc='upper left');
    
# #     plt.show()
#     plt.savefig(PANEL_E, facecolor=plt.gcf().get_facecolor(), transparent=True)  # the figure is saved as svg
#     plt.close()

# svg(SUPP_FIG_SVG)
# !$INKSCAPE --export-pdf $SUPP_FIG_PDF $SUPP_FIG_SVG

### supplementary figure with comparison between LFI and rejection-ABC

In [ ]:
# TODO

### compose figure

In [ ]:
fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)

yoffset = 2.
xoffset = 0.
xoffset_C = 10.
fig = add_svg(fig, PANEL_A, 0 + xoffset + 2, 0 + yoffset - 2)
fig = add_svg(fig, PANEL_B, 0 + xoffset, ROW_2_HEIGHT_MM + yoffset)
fig = add_svg(fig, PANEL_C, ROW_1_WIDTH_COL_1_MM + xoffset + xoffset_C, 0 + yoffset)
fig = add_svg(fig, PANEL_D, ROW_1_WIDTH_COL_1_MM + xoffset + xoffset_C, ROW_2_HEIGHT_MM + yoffset)

yoffset = 2.3
fig = add_label(fig, 'A', 0, 0 + yoffset)
fig = add_label(fig, 'B', 0, ROW_2_HEIGHT_MM + yoffset)
fig = add_label(fig, 'C', ROW_1_WIDTH_COL_1_MM + xoffset_C, 0 + yoffset)
fig = add_label(fig, 'D', ROW_1_WIDTH_COL_1_MM + xoffset + xoffset_C, ROW_2_HEIGHT_MM + yoffset)


PATH_SVG = PATH_DROPBOX_FIGS + 'fig_l5pc.svg'
fig.save(PATH_SVG)


svg(PATH_SVG)

!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig_l5pc.pdf $PATH_SVG